## Genetic Algorithm

Find the maximum of a 2D function: 

$$f(x) = x^3 - 60x^2 + 900x + 100$$

The first step is to generate our initial population of individuals (the current set of possible solutions).

We will iterate over several generations improving it until we find an acceptable solution.

The first generation is randomly generated.

In [102]:
# Generate population function

import random

def generate_population(size, x_boundaries):
    lower_x_boundary, upper_x_boundary = x_boundaries

    population = []
    for i in range(size):
        individual = {
            "x": random.uniform(lower_x_boundary, upper_x_boundary)
        }
        population.append(individual)

    return population

In [103]:
# Apply fitness function

def apply_function(individual):
    x = individual["x"]
    return x**3 - 60 * x**2 + 900 * x + 100

In [104]:
# Roulette choice function

def choice_by_roulette(sorted_population, fitness_sum, exponent):
    offset = 0
    
    # Find the lowest fitness to determine if an offset is needed
    lowest_fitness = min(apply_function(ind) for ind in sorted_population)
    if lowest_fitness < 0:
        offset = -lowest_fitness
    
    # Apply offset and exponent to scale fitness values
    scaled_fitnesses = [(apply_function(ind) + offset) ** exponent for ind in sorted_population]
    normalized_fitness_sum = sum(scaled_fitnesses)
    
    # Draw a random number between 0 and the sum of scaled fitness values
    draw = random.uniform(0, normalized_fitness_sum)

    accumulated = 0
    for individual, fitness in zip(sorted_population, scaled_fitnesses):
        accumulated += fitness

        if draw <= accumulated:
            return individual

    # If we haven't returned by now, something went wrong, raise an error
    raise ValueError("Failed to select an individual by roulette wheel. This should not happen.")

In [105]:
# Sort, cross, mutate, and make next generation functions

def sort_population_by_fitness(population):
    return sorted(population, key=apply_function)


def crossover(individual_a, individual_b):
    xa = individual_a["x"]

    xb = individual_b["x"]

    return {"x": (xa + xb) / 2}


def mutate(individual, low_bound, high_bound, mutation):
    next_x = individual["x"] + random.uniform(-mutation, mutation)

    lower_boundary, upper_boundary = (low_bound, high_bound)

    # Guarantee we keep inside boundaries
    next_x = min(max(next_x, lower_boundary), upper_boundary)

    return {"x": next_x}


def make_next_generation(previous_population, low_bound, high_bound, mutation, exponent):
    next_generation = []
    sorted_by_fitness_population = sort_population_by_fitness(previous_population)
    population_size = len(previous_population)
    fitness_sum = sum(apply_function(individual) for individual in population)

    for i in range(population_size):
        first_choice = choice_by_roulette(sorted_by_fitness_population, fitness_sum, exponent)
        second_choice = choice_by_roulette(sorted_by_fitness_population, fitness_sum, exponent)

        individual = crossover(first_choice, second_choice)
        individual = mutate(individual, low_bound, high_bound, mutation)
        next_generation.append(individual)

    return next_generation

### Run the Algorithm

In [128]:
# Constants

generations = 100
low_bound = 6
high_bound = 14
population_size = 20
mutation = 0.1
exp = 2
i = 1

In [129]:
# Create initial population

population = generate_population(size=population_size, x_boundaries=(low_bound, high_bound))

In [130]:
# Algorithm

while True:
    print(f"🧬 GENERATION {i}")

    for individual in population:
        print(individual, apply_function(individual))

    if i == generations:
        break

    i += 1

    population = make_next_generation(population, low_bound, high_bound, mutation, exp)

best_individual = sort_population_by_fitness(population)[-1]
print("\n🔬 FINAL RESULT")
print(best_individual, apply_function(best_individual))

🧬 GENERATION 1
{'x': 8.223920229524165} 3999.7636480269302
{'x': 6.93084068594186} 3788.4971537821057
{'x': 7.184926482601561} 3839.9523916233957
{'x': 13.447710614009253} 3784.3806766465077
{'x': 13.756569710920182} 3729.657540735305
{'x': 6.649682561490434} 3725.655128462797
{'x': 10.133952283546492} 4099.464107106506
{'x': 10.239994783810314} 4098.2858982109765
{'x': 10.671640727180957} 4086.7699399903577
{'x': 11.49319373781492} 4036.4404397188828
{'x': 9.705997068310896} 4097.381455340529
{'x': 12.107847807010799} 3976.07454364935
{'x': 8.091507397158578} 3983.7782929371647
{'x': 6.089225240752697} 3581.36321164106
{'x': 11.293107896648692} 4051.998400991615
{'x': 13.773574199243356} 3726.5393096232056
{'x': 11.044861692730603} 4068.3886343719023
{'x': 12.708701711170818} 3899.7619716094023
{'x': 6.600576241024398} 3714.0335238593348
{'x': 10.374623140094393} 4095.8423006341245
🧬 GENERATION 2
{'x': 9.424352190756029} 4089.8681353464053
{'x': 9.412669876086643} 4089.4486963199643
{

In [149]:
# Run trials to determine performance of algorithm

def run_trials(num_trials, generations, population_size, x_boundaries, mutation_rate, exponent):
    final_results = []

    for trial in range(num_trials):
        population = generate_population(size=population_size, x_boundaries=x_boundaries)

        for i in range(generations):
            population = make_next_generation(population, *x_boundaries, mutation_rate, exponent)

        best_individual = sort_population_by_fitness(population)[-1]
        best_fitness = apply_function(best_individual)
        final_results.append(best_fitness)

    # Calculate and print summary statistics
    average_best_fitness = sum(final_results) / num_trials
    min_fitness = min(final_results)
    max_fitness = max(final_results)

    print(f"🔬 SUMMARY STATISTICS AFTER {num_trials} TRIALS:")
    print(f"Average Best Fitness: {average_best_fitness}")
    print(f"Highest Fitness Achieved: {max_fitness}")
    print(f"Lowest Fitness Achieved: {min_fitness}")

    return final_results
    return final_results

In [157]:
# Run the experiment and get statistics

num_trials = 100
x_boundaries = (low_bound, high_bound)

final_results = run_trials(
    num_trials, generations, population_size, x_boundaries, mutation, exp)

🔬 SUMMARY STATISTICS AFTER 100 TRIALS:
Average Best Fitness: 4092.0440692971183
Highest Fitness Achieved: 4099.999888800579
Lowest Fitness Achieved: 4039.129238779462
